## Figure 1:  Co-expressed genes share biological function and regulatory architecture

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import upsetplot as up
from tqdm.auto import tqdm 
from scipy.stats import spearmanr
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
import pyranges as pr

# set higher figure resolution
import matplotlib as mpl
#mpl.rcParams['figure.dpi']= 1000

tqdm.pandas()

# get outputs from a config file
prefix = '/home/klawren/oak/pcqtls'
import yaml
config_path= f'{prefix}/config/proteincoding_main.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

import sys
sys.path.append('/home/klawren/oak/pcqtls/workflow/scripts')
from notebook_helper_functions import *
from annotate_null_clusters import * 
from residualize import calculate_residual

# colors
corr_cmap = LinearSegmentedColormap.from_list('corr', [(0, '#3e8093ff'), (.5, 'white'), (1, '#c4553aff')])
gtex_tissue_dict = {
    "Adipose_Subcutaneous": "#FFA54F",
    "Adipose_Visceral_Omentum": "#EE9A00",
    "Artery_Tibial": "#FF0000",
    "Cells_Cultured_fibroblasts": "#9AC0CD",
    "Esophagus_Mucosa": "#8B7355",
    "Esophagus_Muscularis": "#CDAA7D",
    "Lung": "#9ACD32",
    "Muscle_Skeletal": "#7A67EE",
    "Nerve_Tibial": "#FFD700",
    "Skin_Not_Sun_Exposed_Suprapubic": "#3A5FCD",
    "Skin_Sun_Exposed_Lower_leg": "#1E90FF",
    "Thyroid": "#008B45",
    "Whole_Blood": "#FF00FF"
}

gtex_tissue_abbrev = ['ADPSBQ',
                      'ADPVSC',
                      'ARTTBL',
                      'CELLS',
                      'ESPMCS',
                      'ESPMSL',
                      'LUNG',
                      'MSCLSK',
                      'NERVET',
                      'SKINNS',
                      'SKINS',
                      'THYROID',
                      'WHLBLD']

gtex_tissue_pal = sns.color_palette(list(gtex_tissue_dict.values()))
gtex_tissue_pal_df = pd.DataFrame(pd.Series(gtex_tissue_dict), columns=['hex']).reset_index(names=['tissue_id'])
gtex_tissue_pal

# %config InlineBackend.figure_formats = ['svg']
# %matplotlib inline

## Main figure panels

In [ ]:
# load in clusters
clusters = load_across_tissues(config, load_clusters_annotated)

### number clusters by tissue

In [ ]:
# get total number of clusters and genes in clusters
cluster_counts = clusters.groupby('tissue_id').agg({'N_genes':sum, 'Transcripts':'nunique'})
tissue_ids = load_tissue_ids(config)

# plot showing number of clusters in each tissue
fig, ax = plt.subplots(figsize=(2.5,3.5))
ax = sns.barplot(cluster_counts,hue_order=tissue_ids, x='Transcripts', hue='tissue_id', 
                 y='tissue_id', palette=gtex_tissue_pal, ax=ax, width=1, edgecolor='k')
ax.set_xlim((0, ax.get_xlim()[1]+100))
ax.set_xticks([0,500,1000])
ax.set_xlabel('Clusters', fontsize=12)
ax.set_ylabel('')
ax.spines[['top', 'right']].set_visible(False)

# add numbers to plot
for container in ax.containers:
    ax.bar_label(container, padding=2)

# Replace underscores with tisue abbrevations
ax.set_yticklabels(gtex_tissue_abbrev)

plt.tight_layout() 
plt.show()

### cluster size by correlation sign


In [ ]:
clusters['corr_type'] = np.where(clusters['Mean_neg_cor'].isna(), 'positive only', np.where(clusters['Mean_pos_cor'].isna(), 'negative only', 'mixed'))
clusters['N_genes_clip'] = clusters['N_genes'].clip(0, 20)

# histogram not broken up by cluster size
fig, ax = plt.subplots(figsize=(3.5,3.5))

sns.histplot(clusters, y='N_genes_clip', ax=ax, alpha=.8, discrete=True, hue='corr_type', multiple='dodge', palette=['grey', '#c4553aff', '#3e8093ff'], shrink=.8)
ax.set_xlabel('Clusters', fontsize=12)
ax.set_ylabel('Number of genes in cluster', fontsize=12)
ax.set_ylim([1, 21])
ax.set_yticks(np.arange(2, 21)[::2])
ax.set_yticklabels([*np.arange(2, 20)[::2], '20+'], fontsize=10)
ax.set_xscale('log')
ax.spines[['top', 'right']].set_visible(False)

# customize the legend
legend = ax.get_legend()
legend.get_frame().set_visible(False)
legend.set_title("Gene corelations in cluster:")

plt.tight_layout() 
plt.show()

clusters['N_genes'].clip(0, 4).reset_index().groupby('N_genes').agg({'index':'nunique'})/len(clusters) * 100

### example chr from fibroblast

In [ ]:
my_tissue_id = 'Cells_Cultured_fibroblasts'
chr_num = 7

# load in covatiates
covariates_df = pd.read_csv('{}/{}/{}.v8.covariates.txt'.format(prefix, config['covariates_dir'], my_tissue_id), sep='\t', index_col=0).T
base_covariates = covariates_df[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'pcr', 'platform', 'sex']]
peer_covariates = covariates_df[[f'InferredCov{i}' for i in range(1,61,1)]]

# load in expression
expression_df = load_expression(config, my_tissue_id)
expression_df = expression_df.set_index('gene_id')[covariates_df.index]


# load in the gene information (for gene-gene distances)
gid_gencode, full_gencode = load_gencode()
gid_gencode_tissue = gid_gencode.loc[expression_df.index]
# get the genes to plot
selected_chr_gene_ids = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{chr_num}'].index.values
selected_chr_gene_names = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{chr_num}']['gene_name']

In [ ]:
# function to residualize expression based on two covariate dfs
def two_part_residualize(expression_df, base_covariates_df, additional_covariates_df):
    full_covariates = pd.concat([base_covariates_df, additional_covariates_df], axis=1)
    # residulize the expression 
    residal_exp = calculate_residual(expression_df[full_covariates.index], full_covariates, center=True)
    residal_exp = pd.DataFrame(residal_exp, columns=full_covariates.index, index=expression_df.index)
    return residal_exp


# corr with all 60 peers removed
peer_residual_exp = two_part_residualize(expression_df, base_covariates, peer_covariates)
corrected_corr, corrected_pvalue = spearmanr(peer_residual_exp.loc[selected_chr_gene_ids], axis=1)

# plot a zoomed in part

# for chr 17 skin sun exposed
# start_idx = 417
# end_idx = 554
start_idx = 0
end_idx = len(corrected_corr)

corrected_corr = corrected_corr[start_idx:end_idx, start_idx:end_idx]
corrected_pvalue = corrected_pvalue[start_idx:end_idx, start_idx:end_idx]

# calculate total number of pairs considered for bonferroni correction
total_pairs = 0
max_cluster_size=50
for i in np.arange(1,23,1):
    chr_gene_ids = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{i}'].index.values
    upper_corner_idxs = np.triu(np.ones(len(chr_gene_ids)), k=1)
    excluded_cluster_size_idxs = np.triu(np.ones(len(chr_gene_ids)), k=max_cluster_size)
    total_pairs += upper_corner_idxs.sum()  - excluded_cluster_size_idxs.sum()
    
# a version with non-signifigant correlations blanked out
corrected_corr_masked = corrected_corr.copy()
corrected_corr_masked[corrected_pvalue >= (.05/total_pairs)] = 0
corrected_corr_masked = pd.DataFrame(corrected_corr_masked, index=selected_chr_gene_names[start_idx:end_idx], columns=selected_chr_gene_names[start_idx:end_idx])



fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corrected_corr_masked, mask=np.triu(np.ones_like(corrected_corr), k=1), cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, cbar_kws={'label':'Spearman Corrleation', 'pad':0}, xticklabels=False, yticklabels=False, cbar=False)

# add in clusters
for idx, cluster in clusters[(clusters['Chromosome']==chr_num)&(clusters['tissue_id']==my_tissue_id)].iterrows():
    cluster_idxs = np.concatenate([np.where(selected_chr_gene_ids[start_idx:end_idx]==cluster['Transcripts'].split(',')[i])[0] for i in range(cluster['N_genes'])])
    if len(cluster_idxs) > 0:
        # Calculate rectangle properties
        x = cluster_idxs[0]
        y = cluster_idxs[0]
        width = np.ptp(cluster_idxs) + 1
        height = np.ptp(cluster_idxs) + 1
        ax.plot([x, x + width], [y + height, y + height], color='k', linewidth=1)
        ax.plot([x, x], [y, y + height], color='k', linewidth=1)

        # ax.add_patch(mpatches.Rectangle((cluster_idxs[0], cluster_idxs[0]), 
        #                             width=np.ptp(cluster_idxs)+1, height=np.ptp(cluster_idxs)+1,
        #                             linewidth=1, edgecolor='k', facecolor='none'))
#ax.spines[['bottom', 'left']].set_visible(True)
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

### example chr from skin

In [ ]:
# load in expression and covariates
my_tissue_id = 'Skin_Sun_Exposed_Lower_leg'
chr_num = 17


# load in covatiates
covariates_df = pd.read_csv('{}/{}/{}.v8.covariates.txt'.format(prefix, config['covariates_dir'], my_tissue_id), sep='\t', index_col=0).T
base_covariates = covariates_df[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'pcr', 'platform', 'sex']]
peer_covariates = covariates_df[[f'InferredCov{i}' for i in range(1,61,1)]]

# load in expression
expression_df = load_expression(config, my_tissue_id)
expression_df = expression_df.set_index('gene_id')[covariates_df.index]


# load in the gene information (for gene-gene distances)
gid_gencode, full_gencode = load_gencode()
gid_gencode_tissue = gid_gencode.loc[expression_df.index]
# get the genes to plot
selected_chr_gene_ids = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{chr_num}'].index.values
selected_chr_gene_names = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{chr_num}']['gene_name']

In [ ]:
# function to residualize expression based on two covariate dfs
def two_part_residualize(expression_df, base_covariates_df, additional_covariates_df):
    full_covariates = pd.concat([base_covariates_df, additional_covariates_df], axis=1)
    # residulize the expression 
    residal_exp = calculate_residual(expression_df[full_covariates.index], full_covariates, center=True)
    residal_exp = pd.DataFrame(residal_exp, columns=full_covariates.index, index=expression_df.index)
    return residal_exp


# corr with all 60 peers removed
peer_residual_exp = two_part_residualize(expression_df, base_covariates, peer_covariates)
corrected_corr, corrected_pvalue = spearmanr(peer_residual_exp.loc[selected_chr_gene_ids], axis=1)

# plot a zoomed in part

# for chr 17 skin sun exposed
# start_idx = 417
# end_idx = 554
start_idx = 0
end_idx = len(corrected_corr)

corrected_corr = corrected_corr[start_idx:end_idx, start_idx:end_idx]
corrected_pvalue = corrected_pvalue[start_idx:end_idx, start_idx:end_idx]

# calculate total number of pairs considered for bonferroni correction
total_pairs = 0
max_cluster_size=50
for i in np.arange(1,23,1):
    chr_gene_ids = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{i}'].index.values
    upper_corner_idxs = np.triu(np.ones(len(chr_gene_ids)), k=1)
    excluded_cluster_size_idxs = np.triu(np.ones(len(chr_gene_ids)), k=max_cluster_size)
    total_pairs += upper_corner_idxs.sum()  - excluded_cluster_size_idxs.sum()
    
# a version with non-signifigant correlations blanked out
corrected_corr_masked = corrected_corr.copy()
corrected_corr_masked[corrected_pvalue >= (.05/total_pairs)] = 0
corrected_corr_masked = pd.DataFrame(corrected_corr_masked, index=selected_chr_gene_names[start_idx:end_idx], columns=selected_chr_gene_names[start_idx:end_idx])



fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corrected_corr_masked, mask=np.triu(np.ones_like(corrected_corr), k=1), cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, cbar_kws={'label':'Spearman Corrleation', 'pad':0}, xticklabels=False, yticklabels=False, cbar=False)

# add in clusters
for idx, cluster in clusters[(clusters['Chromosome']==chr_num)&(clusters['tissue_id']==my_tissue_id)].iterrows():
    cluster_idxs = np.concatenate([np.where(selected_chr_gene_ids[start_idx:end_idx]==cluster['Transcripts'].split(',')[i])[0] for i in range(cluster['N_genes'])])
    if len(cluster_idxs) > 0:
        # Calculate rectangle properties
        x = cluster_idxs[0]
        y = cluster_idxs[0]
        width = np.ptp(cluster_idxs) + 1
        height = np.ptp(cluster_idxs) + 1
        ax.plot([x, x + width], [y + height, y + height], color='k', linewidth=1)
        ax.plot([x, x], [y, y + height], color='k', linewidth=1)

        # ax.add_patch(mpatches.Rectangle((cluster_idxs[0], cluster_idxs[0]), 
        #                             width=np.ptp(cluster_idxs)+1, height=np.ptp(cluster_idxs)+1,
        #                             linewidth=1, edgecolor='k', facecolor='none'))
#ax.spines[['bottom', 'left']].set_visible(True)
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

### pulled out example chromosomes

In [ ]:
def plot_cluster(example_cluster, cbar=False):
    # gene gene ids sorted by tss start
    sorted_cluster = gid_gencode.loc[example_cluster['cluster_id'].split('_')].sort_values('tss_start')
    sorted_gene_ids = sorted_cluster.index.values
    # get normed expression 
    expression = load_cluster_expression(config, example_cluster['Tissue'])
    cluster_expression = expression[(expression['cluster_id']==example_cluster['cluster_id'])].set_index('egene_id').loc[sorted_gene_ids]
    # get correlation of expression
    sample_ids = cluster_expression.columns[cluster_expression.columns.str.contains('GTEX')]
    cluster_corr, cluster_pvalue = spearmanr(cluster_expression[sample_ids], axis=1)
    # make df wtih readable gene names
    cluster_corr = pd.DataFrame(cluster_corr, index=sorted_cluster['gene_name'].values, columns=sorted_cluster['gene_name'].values)

    # make corr plot
    
    fig_size = int(len(cluster_corr)/2)
    print(fig_size)
    fig, ax = plt.subplots(figsize=(fig_size, fig_size))
    sns.heatmap(cluster_corr, mask=np.triu(np.ones_like(cluster_corr), k=1),
                cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, 
                cbar_kws={'label':'Spearman Corrleation'}, 
                xticklabels=True, yticklabels=True, cbar=cbar)
    ax.tick_params("y", rotation=0) 
    plt.show()

In [ ]:
plot_cluster(clusters.iloc[3132], cbar=True)

In [ ]:
plot_cluster(clusters.iloc[9757])

### cluster enrichments

In [ ]:
# get just the ones with matched enahncer
enhancer_tissue_ids = ['Artery_Tibial', 'Cells_Cultured_fibroblasts', 'Lung', 'Muscle_Skeletal', 'Skin_Not_Sun_Exposed_Suprapubic', 'Skin_Sun_Exposed_Lower_leg', 'Thyroid']
combined_clusters = load_across_tissues(config, load_clusters_annotated, tissue_ids = enhancer_tissue_ids)
def load_all_multigene_null(config, tissue_id):
    return pd.concat([load_null_clusters_annotated(config, tissue_id, num_genes) for num_genes in [2,3,4,5]])
combined_multigene_nulls =  load_across_tissues(config, load_all_multigene_null, tissue_ids = enhancer_tissue_ids)

# exclude highly cross mappable genes and add columns
combined_clusters = combined_clusters[~combined_clusters['has_cross_map']]
combined_clusters['has_multiple_abc_genes'] = combined_clusters['num_abc_genes'] > 1
combined_clusters['log_size'] = np.log10(combined_clusters['cluster_tss_size'])
combined_multigene_nulls = combined_multigene_nulls[~combined_multigene_nulls['has_cross_map']]
combined_multigene_nulls['has_multiple_abc_genes'] = combined_multigene_nulls['num_abc_genes'] > 1
combined_multigene_nulls['log_size'] = np.log10(combined_multigene_nulls['cluster_tss_size'])


# make nulls  

# make one without distance matching
multitissue_abc_largerclusters_df = pd.concat([combined_clusters, combined_multigene_nulls], keys=['cluster', 'null'], names=['type', 'idx']).reset_index()
multitissue_abc_largerclusters_df['is_cluster'] = multitissue_abc_largerclusters_df['type']=='cluster'

# make one with distance matching
multitissue_abc_resamp_largerclusters_nulls = []
for num_genes in combined_multigene_nulls['N_genes'].unique():
    cluster_num_genes = combined_clusters[(combined_clusters['N_genes']==num_genes)]
    # don't bother to match if too few clusters of this size, just discard null
    if len(cluster_num_genes) > 2:
        null_num_genes = combined_multigene_nulls[(combined_multigene_nulls['N_genes']==num_genes)]
        multitissue_abc_resamp_largerclusters_nulls.append(get_resamp_null_cluster(null_num_genes, cluster_num_genes, number_null=5000*len(enhancer_tissue_ids), plot=False))

multitissue_abc_resamp_largerclusters_df = pd.concat([combined_clusters, pd.concat(multitissue_abc_resamp_largerclusters_nulls)], keys=['cluster', 'null'], names=['type', 'idx']).reset_index()
multitissue_abc_resamp_largerclusters_df['is_cluster'] = multitissue_abc_resamp_largerclusters_df['type']=='cluster'

In [ ]:
multitissue_abc_largerclusters_df['corr_type'] = np.where(multitissue_abc_largerclusters_df['Mean_neg_cor'].isna(), 'positive_only', np.where(multitissue_abc_largerclusters_df['Mean_pos_cor'].isna(), 'negative_only', 'mixed'))


In [ ]:
# get odds df for positive negative, and mixed corr clusters

multitissue_abc_largerclusters_df['corr_type'] = np.where(multitissue_abc_largerclusters_df['Mean_neg_cor'].isna(), 'positive_only', np.where(multitissue_abc_largerclusters_df['Mean_pos_cor'].isna(), 'negative_only', 'mixed'))

noresamp_cols = ['has_bidirectional_promoter']
noresamp_odds_positive = get_odds_df(multitissue_abc_largerclusters_df[((multitissue_abc_largerclusters_df['type']=='cluster')&(multitissue_abc_largerclusters_df['corr_type']=='positive_only')) | (multitissue_abc_largerclusters_df['type']=='null')], column_list = noresamp_cols, correct_on=True, correct_on_column='N_genes', label_col='is_cluster')
noresamp_odds_all = get_odds_df(multitissue_abc_largerclusters_df[((multitissue_abc_largerclusters_df['type']=='cluster')&(multitissue_abc_largerclusters_df['corr_type']=='mixed')) | (multitissue_abc_largerclusters_df['type']=='null')], column_list = noresamp_cols, correct_on=True, correct_on_column='N_genes', label_col='is_cluster')
noresamp_odds_negative = get_odds_df(multitissue_abc_largerclusters_df[((multitissue_abc_largerclusters_df['type']=='cluster')&(multitissue_abc_largerclusters_df['corr_type']=='negative_only')) | (multitissue_abc_largerclusters_df['type']=='null')], column_list = noresamp_cols, correct_on=True, correct_on_column='N_genes', label_col='is_cluster')


multitissue_abc_resamp_largerclusters_df['corr_type'] = np.where(multitissue_abc_resamp_largerclusters_df['Mean_neg_cor'].isna(), 'positive_only', np.where(multitissue_abc_resamp_largerclusters_df['Mean_pos_cor'].isna(), 'negative_only', 'mixed'))

resamp_column_list = ['has_tads_tss', 
                      'has_paralog', 
                      'has_shared_go_any',
                      'has_ctcf_peak',
                      'has_shared_enhancer']

bool_filter_list = ['has_shared_enhancer']

resamp_odds_positive = get_odds_df(multitissue_abc_resamp_largerclusters_df[((multitissue_abc_resamp_largerclusters_df['type']=='cluster')&(multitissue_abc_resamp_largerclusters_df['corr_type']=='positive_only')) | (multitissue_abc_resamp_largerclusters_df['type']=='null')], column_list=resamp_column_list, filter_list=bool_filter_list, correct_on=True, correct_on_column='N_genes', label_col='is_cluster') 
resamp_odds_all = get_odds_df(multitissue_abc_resamp_largerclusters_df[((multitissue_abc_resamp_largerclusters_df['type']=='cluster')&(multitissue_abc_resamp_largerclusters_df['corr_type']=='mixed')) | (multitissue_abc_resamp_largerclusters_df['type']=='null')], column_list=resamp_column_list, filter_list=bool_filter_list, correct_on=True, correct_on_column='N_genes', label_col='is_cluster')
resamp_odds_negative = get_odds_df(multitissue_abc_resamp_largerclusters_df[((multitissue_abc_resamp_largerclusters_df['type']=='cluster')&(multitissue_abc_resamp_largerclusters_df['corr_type']=='negative_only')) | (multitissue_abc_resamp_largerclusters_df['type']=='null')], column_list=resamp_column_list, filter_list=bool_filter_list, correct_on=True, correct_on_column='N_genes', label_col='is_cluster')


In [ ]:
# combined, resampled and nonresampled columns
col_order = ['has_tads_tss', 'has_ctcf_peak', 'has_shared_enhancer', 'has_bidirectional_promoter', 'has_shared_go_any', 'has_paralog'][::-1]
odds_negative = pd.concat([noresamp_odds_negative, resamp_odds_negative]).loc[col_order] 
odds_all = pd.concat([noresamp_odds_all, resamp_odds_all]).loc[col_order] 
odds_positive = pd.concat([noresamp_odds_positive, resamp_odds_positive]).loc[col_order] 

fig, ax = plt.subplots(figsize=(3.5, 3.5))

# make the plot
make_log_odds_plot_multiple([odds_negative, odds_positive], 
                             labels=['negative', 'positive',], 
                             colors = sns.color_palette('blend:#67AFD2,k,#B83A4B', n_colors=2), 
                             add_annotations=False, offset=.3, ax=ax)

ax.set_xlabel(r'$log_{10}$(Fold enrichment correlated clusters)', fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_yticklabels([])
ax.set_xlim([.4, 11])
ax.set_xticks([.5, 1, 5, 10], labels=[.5, 1, 5, 10])

# customize the legend
legend = ax.get_legend()
legend.get_frame().set_visible(False)
legend.set_title("Gene correlaitons in cluster:")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1.1, 1))


In [ ]:
# Calculate the fraction of True values for each column in col_order
fraction_true = combined_clusters[col_order].mean()

# Create a vertical bar graph
plt.figure(figsize=(2.5, 3.5))
ax = sns.barplot(y=fraction_true.index, x=fraction_true.values, color='grey')
ax.bar_label(ax.containers[0], fmt='%.2f')
ax.set_ylabel(" ")
ax.set_xlabel("Proportion of clusters", fontsize=12)
ax.set_xlim([0,1])
ax.set_yticklabels(np.array(['paralogs',  'shared GO',  'promoter', 'Enhancer',  'CTCF',  'crosses TAD'][::-1]), fontsize=12)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


### a) chromosome correlaiton before and after peer correction

In [ ]:
# calculate total number of pairs considered for bonferroni correction
total_pairs = 0
max_cluster_size=50
for i in np.arange(1,23,1):
    chr_gene_ids = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{i}'].index.values
    upper_corner_idxs = np.triu(np.ones(len(chr_gene_ids)), k=1)
    excluded_cluster_size_idxs = np.triu(np.ones(len(chr_gene_ids)), k=max_cluster_size)
    total_pairs += upper_corner_idxs.sum()  - excluded_cluster_size_idxs.sum()

In [ ]:
# inital corr with no peer factors removed
residal_exp = calculate_residual(expression_df[base_covariates.index], base_covariates, center=True)
residal_exp = pd.DataFrame(residal_exp, columns=base_covariates.index, index=expression_df.index)
inital_corr, initial_pvalue = spearmanr(residal_exp.loc[selected_chr_gene_ids], axis=1)

# a version with non-signifigant correlations blanked out
inital_corr_masked = inital_corr.copy()
inital_corr_masked[initial_pvalue >= (.05/total_pairs)] = 0
inital_corr_masked = pd.DataFrame(inital_corr_masked, index=selected_chr_gene_names, columns=selected_chr_gene_names)

fig, ax = plt.subplots(figsize=(5, 5))
sns.heatmap(inital_corr_masked, mask=np.triu(np.ones_like(inital_corr), k=1), cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, cbar_kws={'label':'Spearman Corrleation', 'pad':0}, xticklabels=False, yticklabels=False, cbar=False)
ax.spines[['bottom', 'left']].set_visible(True)
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

In [ ]:
# function to residualize expression based on two covariate dfs
def two_part_residualize(expression_df, base_covariates_df, additional_covariates_df):
    full_covariates = pd.concat([base_covariates_df, additional_covariates_df], axis=1)
    # residulize the expression 
    residal_exp = calculate_residual(expression_df[full_covariates.index], full_covariates, center=True)
    residal_exp = pd.DataFrame(residal_exp, columns=full_covariates.index, index=expression_df.index)
    return residal_exp


# corr with all 60 peers removed
peer_residual_exp = two_part_residualize(expression_df, base_covariates, peer_covariates)
corrected_corr, corrected_pvalue = spearmanr(peer_residual_exp.loc[selected_chr_gene_ids], axis=1)

# plot a zoomed in part

# for chr 17 skin sun exposed
# start_idx = 417
# end_idx = 554
start_idx = 0
end_idx = len(corrected_corr)

corrected_corr = corrected_corr[start_idx:end_idx, start_idx:end_idx]
corrected_pvalue = corrected_pvalue[start_idx:end_idx, start_idx:end_idx]


# a version with non-signifigant correlations blanked out
corrected_corr_masked = corrected_corr.copy()
corrected_corr_masked[corrected_pvalue >= (.05/total_pairs)] = 0
corrected_corr_masked = pd.DataFrame(corrected_corr_masked, index=selected_chr_gene_names[start_idx:end_idx], columns=selected_chr_gene_names[start_idx:end_idx])



fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corrected_corr_masked, mask=np.triu(np.ones_like(corrected_corr), k=1), cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, cbar_kws={'label':'Spearman Corrleation', 'pad':0}, xticklabels=False, yticklabels=False, cbar=False)

# add in clusters
for idx, cluster in clusters[(clusters['Chromosome']==chr_num)&(clusters['tissue_id']==my_tissue_id)].iterrows():
    cluster_idxs = np.concatenate([np.where(selected_chr_gene_ids[start_idx:end_idx]==cluster['Transcripts'].split(',')[i])[0] for i in range(cluster['N_genes'])])
    if len(cluster_idxs) > 0:
        # Calculate rectangle properties
        x = cluster_idxs[0]
        y = cluster_idxs[0]
        width = np.ptp(cluster_idxs) + 1
        height = np.ptp(cluster_idxs) + 1
        ax.plot([x, x + width], [y + height, y + height], color='k', linewidth=1)
        ax.plot([x, x], [y, y + height], color='k', linewidth=1)

        # ax.add_patch(mpatches.Rectangle((cluster_idxs[0], cluster_idxs[0]), 
        #                             width=np.ptp(cluster_idxs)+1, height=np.ptp(cluster_idxs)+1,
        #                             linewidth=1, edgecolor='k', facecolor='none'))
#ax.spines[['bottom', 'left']].set_visible(True)
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

### b) quantification of correlaiton distances before and after peer correction

In [ ]:
def get_signifigant_distances(gid_gencode_tissue, expression_df, base_covariates, peer_covariates, chr_num, residualization_values  = [0,15,30,45,60]):
    selected_chr_gene_ids = gid_gencode_tissue[gid_gencode_tissue['chr'] == f'chr{chr_num}'].index.values
    # df with pairswise distances
    start_positions = gid_gencode_tissue.loc[selected_chr_gene_ids]['tss_start'].values
    dist_matrix = np.abs(start_positions[:, None] - start_positions)

    # distance of signifigant corrs as pc/peers residualized away? 
    signifigant_distances = {}
    for i in residualization_values:
        # expression residualized with some pcs
        sub_exp = two_part_residualize(expression_df, base_covariates, peer_covariates.iloc[:, :i])
        pc_residual_corr, pc_residual_pvalue = spearmanr(sub_exp.loc[selected_chr_gene_ids], axis=1)
        pc_residual_pvalue[np.triu_indices(len(pc_residual_pvalue))] = 1
        signifigant_distances[i] = (dist_matrix[pc_residual_pvalue<(.05/total_pairs)])
    return signifigant_distances

In [ ]:
# loop through all tissues, takes ~ 5 minutes
all_chr_distances = []
for my_tissue_id in load_tissue_ids(config):
    print(my_tissue_id)
    covariates_df = pd.read_csv('{}/{}/{}.v8.covariates.txt'.format(prefix, config['covariates_dir'], my_tissue_id), sep='\t', index_col=0).T
    base_covariates = covariates_df[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'pcr', 'platform', 'sex']]
    peer_covariates = covariates_df[[f'InferredCov{i}' for i in range(1,61,1)]]
    expression_df = load_expression(config, my_tissue_id)
    expression_df = expression_df.set_index('gene_id')[covariates_df.index]
    gid_gencode_tissue =  gid_gencode.loc[expression_df.index]

    # loop through chroms
    for i in tqdm(range(1, 23), total=23):
        all_chr_distances.append(get_signifigant_distances(gid_gencode_tissue, expression_df, base_covariates, peer_covariates, 
                                                        i, residualization_values  = [0,60]))

In [ ]:
# combine over chroms and tissues
combined_chr_distances = {}
for i in [0,60]:
    combined_chr_distances[i] = np.concatenate([chr_distances[i] for chr_distances in all_chr_distances])

In [ ]:
mpl.rcParams['figure.dpi']= 1000
fig, ax = plt.subplots(figsize=(4,5))
sns.violinplot(combined_chr_distances, ax=ax, color='grey', fill=False, cut=0)
ax.set_ylabel('Correlated genes\' distance\' (bp)', fontsize=12)
ax.set_xticklabels(['Unresidualized', 'Residualized'], fontsize=12)
ax.spines[['top', 'right']].set_visible(False)
plt.show()
mpl.rcParams['figure.dpi']= 100

### c) examples of clusters

### fraction of genes in clusters

In [ ]:
# load in expression data (so we only compare to pairs of genes also expressed in this tissue)
# load in expression data for all tissues 
# takes about 30 seconds
expression={}
for tissue_id in tissue_ids: 
    expression[tissue_id] = load_expression(config, tissue_id)
    expression[tissue_id] = expression[tissue_id].sort_values(['#chr', 'start', 'end'])
    # we only need the transcript ids from the expression dir 
    expression[tissue_id].drop(columns=expression[tissue_id].columns[4:], inplace=True)
    
expression_df = pd.concat(expression, keys=tissue_ids, names=('tissue_id', 'idx')).reset_index('idx', drop=True)

In [ ]:
frac_genes_in_clusters=[]
for tissue_id in tissue_ids: 
    all_cluster_transcripts = np.concatenate(clusters[clusters['tissue_id'] == tissue_id]['Transcripts'].str.split(',').values)
    frac_genes_in_clusters.append(sum(expression[tissue_id]['gene_id'].isin(all_cluster_transcripts))/len(expression[tissue_id])*100)

frac_df = pd.DataFrame({'tissue_id': tissue_ids, 'fraction':frac_genes_in_clusters})
print(frac_df['fraction'].mean())
print(frac_df['fraction'].min())
print(frac_df['fraction'].max())

# plot
fig, ax = plt.subplots(figsize=(5,5))

sns.barplot(pd.DataFrame({'tissue_id': tissue_ids, 'fraction':frac_genes_in_clusters}), x='fraction', y='tissue_id', palette=gtex_tissue_pal, 
            ax =ax, saturation=1, edgecolor='k', width=1)
ax.set_xlabel('Fraction expressed genes in clusters')
ax.set_ylabel('')
ax.spines[['top', 'right']].set_visible(False)

# add numbers to plot
for container in ax.containers:
    ax.bar_label(container, padding=2, fmt=f'%.1f')

# Replace underscores with tisue abbrevations
ax.set_yticklabels(gtex_tissue_abbrev)

plt.tight_layout() 
plt.show()
plt.show()

### gene orientation/ relative position
 also the .genes. gtf does not necessaritly have accurate start/end positions for genes due to collapse



types of genes (heirarchy so top ones are annotated even if bottom ones are true too)
* shared_promoter (promoter defined 2000 bp upsteam of transcript start to 200 bp downstream of transcript start) same or opp orientiaton 
* convergent/divergent/tandem (no promoter overlap, with or without _overlapping tag for convergent and tandem)
* any transcirpt overlap? 

In [ ]:
gencode_v26=pr.read_gtf('/home/klawren/oak/pcqtls/data/references/gencode.v26.annotation.gtf.gz').as_df()

# this is all done at the transcript level
gencode_v26 = gencode_v26[gencode_v26['Feature'] == 'transcript'].set_index('gene_id')

In [ ]:
gencode_v26['tss_start'] =  np.where(gencode_v26['Strand'] == '+', gencode_v26['Start'], gencode_v26['End'])
gencode_v26['promoter_start'] = np.where(gencode_v26['Strand'] == '+', gencode_v26['tss_start'] - 1000, gencode_v26['tss_start'] - 100)
gencode_v26['promoter_end'] = np.where(gencode_v26['Strand'] == '+', gencode_v26['tss_start'] + 100, gencode_v26['tss_start'] + 1000)

In [ ]:
# make clusters have a row per gene comparision that we are doing
clusters['gene_id_1'] = clusters['Transcripts'].str.split(',')
clusters['gene_id_2'] = clusters['Transcripts'].str.split(',')
clusters_explode = clusters.explode('gene_id_1').explode('gene_id_2')
clusters_explode = clusters_explode[~(clusters_explode['gene_id_1'] == clusters_explode['gene_id_2'])]
clusters_explode['gene_id_pair'] = clusters_explode['gene_id_1'] + '_' + clusters_explode['gene_id_2']

In [ ]:
row = clusters.iloc[19]
gene_1 = row['Transcripts'].split(',')[0]
gene_2 = row['Transcripts'].split(',')[1]
gene_1_gencode = gencode_v26.loc[gene_1]
gene_2_gencode = gencode_v26.loc[gene_2]

In [ ]:
gene_1_gencode

In [ ]:
gene_2_gencode

In [ ]:
gene_1_gencode['tss_start'].max() > gene_2_gencode['tss_start'].max()

In [ ]:
gene_1_gencode['Strand'][0]=='+', gene_1_gencode['tss_start'].max() > gene_2_gencode['tss_start'].max()

In [ ]:
get_gene_relative_position(gene_1_gencode, gene_2_gencode)

In [ ]:
def get_gene_relative_position(gene_1_gencode, gene_2_gencode):
    if (gene_1_gencode['Strand'][0] == gene_2_gencode['Strand'][0]) > 0:
        orientaiton = 'same'
    else:
        orientaiton = 'opp'

    # check for shared promoter
    overlapping_promoter=False
    overlapping_transcript=False

    try:
        for idx1, gene_1_row in gene_1_gencode.iterrows():
            if (((gene_1_row['promoter_start'] < gene_2_gencode['promoter_end']) & (gene_1_row['promoter_start'] > gene_2_gencode['promoter_start'])).any() |
                ((gene_1_row['promoter_end'] < gene_2_gencode['promoter_end']) & (gene_1_row['promoter_end'] > gene_2_gencode['promoter_start'])).any()):
                overlapping_promoter=True
            if (((gene_1_row['Start'] < gene_2_gencode['End']) & (gene_1_row['Start'] > gene_2_gencode['Start'])).any() |
                ((gene_1_row['End'] < gene_2_gencode['End']) & (gene_1_row['End'] > gene_2_gencode['Start'])).any()):
                overlapping_transcript=True
    except AttributeError:
        gene_1_row = gene_1_gencode
        if (((gene_1_row['promoter_start'] < gene_2_gencode['promoter_end']) & (gene_1_row['promoter_start'] > gene_2_gencode['promoter_start'])).any() |
        ((gene_1_row['promoter_end'] < gene_2_gencode['promoter_end']) & (gene_1_row['promoter_end'] > gene_2_gencode['promoter_start'])).any()):
            overlapping_promoter=True
        if (((gene_1_row['Start'] < gene_2_gencode['End']) & (gene_1_row['Start'] > gene_2_gencode['Start'])).any() |
            ((gene_1_row['End'] < gene_2_gencode['End']) & (gene_1_row['End'] > gene_2_gencode['Start'])).any()):
            overlapping_transcript=True


    print(orientaiton)
    print(overlapping_promoter)
    print(overlapping_transcript)

    if overlapping_promoter:
        if orientaiton=='same':
            return 'shared_promoter'
        elif orientaiton=='opp':
            return 'bidirectional_promoter'
    elif overlapping_transcript:
        if orientaiton=='same':
            return 'tandem_overlapping'
        elif orientaiton=='opp':
            return 'convergent_overlapping'
    else:
        if orientaiton=='same':
            return 'tandem'
        elif gene_1_gencode['Strand'][0]=='+':
            if gene_1_gencode['tss_start'].max() > gene_2_gencode['tss_start'].max():
                return 'divergent'
            else:
                return 'convergent'
        else:
            if gene_1_gencode['tss_start'].max() > gene_2_gencode['tss_start'].max():
                return 'convergent'
            else:
                return 'divergent'



In [ ]:
# start with most lenient annotations then back fill

# convergent non overlapping
clusters['']


# Supplementary figures

### correlaitons before and after correciton with all showing, not just signifigant

In [ ]:
# heatmap corr without non-signifignat blanked out 
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(inital_corr, mask=np.triu(np.ones_like(inital_corr), k=1), cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, cbar_kws={'label':'Spearman Corrleation', 'pad':0}, xticklabels=False, yticklabels=False, cbar=False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

# for supp: version without non-signifignat masked
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(corrected_corr, mask=np.tril(np.ones_like(corrected_corr), k=-1), cmap=corr_cmap, vmin=-1, vmax=1, ax=ax, cbar_kws={'label':'Spearman Corrleation', 'pad':0}, xticklabels=False, yticklabels=False, cbar=False)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

### PCs vs PEERs correlation

In [ ]:
# get pc covariantes (these will be cacluated after the genotype pc and other base covars are residualized)

base_residual_expression = calculate_residual(expression_df[base_covariates.index], base_covariates, center=True)
base_residual_expression = pd.DataFrame(base_residual_expression, columns=base_covariates.index, index=expression_df.index)

# get global pcs 
pca = PCA()
global_pc_values = pca.fit_transform(base_residual_expression.T)
pc_covariates_df = pd.DataFrame(global_pc_values, index=covariates_df.index, columns=[f'PC{i}' for i in range(global_pc_values.shape[0])])

# compare pc values to peer values
pc_peer_corr=np.zeros((60,60))
for i in range(60):
    for j in range(60):
        pc_peer_corr[i,j] = spearmanr(peer_covariates.T.iloc[i], pc_covariates_df.T.iloc[j]).statistic

blue_cmap = LinearSegmentedColormap.from_list('mycmap', [(0, 'white'), (1, '#4298B5')])

ax = sns.heatmap(np.square(pc_peer_corr), vmin=0, vmax=1, cmap=blue_cmap, cbar_kws={'label':'Spearman $r^2$'})
plt.xlabel('PEER factor index')
plt.ylabel('PCA factor index')
ax.set_xticks(ticks = [0, 14, 29, 44, 59], labels=[1, 15, 30, 45, 60])
ax.set_yticks(ticks = [0, 14, 29, 44, 59], labels=[1, 15, 30, 45, 60])
ax.invert_yaxis()

ax.spines[['top', 'bottom', 'left', 'right']].set_visible(True)

### PCs vs PEERs long and short range correlation

In [ ]:
def get_frac_sig_at_decile(residual_pvalue, decile_df, short_decile_cutoff=1, long_decile_cutoff=8):
      fraction_close_sig = []
      fraction_far_sig = []


      # sig corrs by decile
      close_pvals = residual_pvalue[decile_df<short_decile_cutoff]
      fraction_close_sig = sum(close_pvals < (.05/total_pairs))/len(close_pvals)

      # signifigant corrs off the diagonal
      #far_pvals = peer_residual_pvalue[np.triu_indices(len(peer_residual_pvalue), k=3)]

      # sig corrs by decile
      far_pvals = residual_pvalue[decile_df>long_decile_cutoff]
      fraction_far_sig = ((far_pvals < (.05/total_pairs))/len(far_pvals)).sum()

      return fraction_close_sig, fraction_far_sig

def compare_peer_pc(expression_df, base_covariates_df, pc_covariates_df, peer_covariates_df, decile_df, use_mask=False, mask=None):

   fraction_close_sig_pc = []
   fraction_far_sig_pc = []

   fraction_close_sig_peer = []
   fraction_far_sig_peer = []

   for i in range(60):
      # expression residualized with some pcs
      sub_exp = two_part_residualize(expression_df, base_covariates_df, pc_covariates_df.iloc[:, :i])
      pc_residual_corr, pc_residual_pvalue = spearmanr(sub_exp.loc[selected_chr_gene_ids], axis=1)
      np.fill_diagonal(pc_residual_pvalue, 1)
      if use_mask:
         pc_residual_pvalue = np.where(mask, 1, pc_residual_pvalue)

      fraction_close_sig, fraction_far_sig = get_frac_sig_at_decile(pc_residual_pvalue, decile_df)

      fraction_close_sig_pc.append(fraction_close_sig)
      fraction_far_sig_pc.append(fraction_far_sig)

      # expression residualized with some peers
      sub_exp = two_part_residualize(expression_df, base_covariates_df, peer_covariates_df.iloc[:, :i])
      peer_residual_corr, peer_residual_pvalue = spearmanr(sub_exp.loc[selected_chr_gene_ids], axis=1)
      np.fill_diagonal(peer_residual_pvalue, 1)

      if use_mask:
         peer_residual_pvalue = np.where(mask, 1, peer_residual_pvalue)

      fraction_close_sig, fraction_far_sig = get_frac_sig_at_decile(peer_residual_pvalue, decile_df)
      
      fraction_close_sig_peer.append(fraction_close_sig)
      fraction_far_sig_peer.append(fraction_far_sig)

   if use_mask:
      fraction_corr_sig_df = pd.DataFrame({'(mask) PC: short-range correlations':fraction_close_sig_pc, 
                                     '(mask) PC: long-range correlations':fraction_far_sig_pc, 
                                     '(mask) PEER: short-range correlations':fraction_close_sig_peer, 
                                     '(mask) PEER: long-range correlations':fraction_far_sig_peer})

   else:
      fraction_corr_sig_df = pd.DataFrame({'PC: short-range correlations':fraction_close_sig_pc, 
                                     'PC: long-range correlations':fraction_far_sig_pc, 
                                     'PEER: short-range correlations':fraction_close_sig_peer, 
                                     'PEER: long-range correlations':fraction_far_sig_peer})
   return fraction_corr_sig_df 

# matrix of gene-gene distances
start_positions = gid_gencode.loc[selected_chr_gene_ids]['tss_start'].values
dist_matrix = np.abs(start_positions[:, None] - start_positions)

# Create deciles based on distance 
deciles = np.percentile(dist_matrix, [10, 20, 30, 40, 50, 60, 70, 80, 90])
decile_df = pd.DataFrame(np.digitize(dist_matrix, deciles), index=selected_chr_gene_ids, columns=selected_chr_gene_ids)

fraction_corr_sig = compare_peer_pc(expression_df, base_covariates, pc_covariates_df, peer_covariates, decile_df).reset_index()

In [ ]:
fraction_corr_sig_melt = pd.melt(fraction_corr_sig, value_name='frac_corr', id_vars='index')
fraction_corr_sig_melt['Factor type'] = fraction_corr_sig_melt['variable'].str.split(':').str[0]
#fraction_corr_sig_melt['Gene-gene distance'] = fraction_corr_sig_melt['variable'].str.split(': ').str[1].str.split('-').str[0]
fraction_corr_sig_melt['Gene-gene distance'] = np.where(fraction_corr_sig_melt['variable'].str.contains('short'), 'Short (first decile)', 'Long (last decile)')

fig, ax = plt.subplots(figsize=(7,4))
sns.scatterplot(fraction_corr_sig_melt, x='index', y='frac_corr', ax=ax, hue='Factor type',
                     style='Gene-gene distance', palette=('#4298B5', '#6FA287'), alpha=.8, edgecolor=None)

ax.set_xlabel('Number of global factors residualized')
ax.set_ylabel('Fraction signifigant correlations')
ax.set_xticks(ticks = [0, 14, 29, 44, 59], labels=[1, 15, 30, 45, 60])
ax.spines[['top', 'right']].set_visible(False)
ax.set_yscale('log')
ax.set_yticks(ticks = [.5, .1, .01, .001], labels=[1/2, 1/10, 1/100, 1/1000])

plt.gca().get_legend().set_frame_on(False)
plt.show()

### shuffled gene locations null

In [ ]:
import call_clusters
real_cluster_df = load_cluster(config, my_tissue_id)
expression_df_full = load_expression(config, my_tissue_id)

# cluster density on per chrom basis for the actual clusters
cluster_density = []
for i in np.arange(1,23,1):
    chr_gene_ids = expression_df_full[expression_df_full['#chr'] == f'chr{i}']['gene_id']
    cluster_density.append(real_cluster_df[real_cluster_df['Chromosome'] == i]['N_genes'].sum()/len(chr_gene_ids))


# randomly resample the dataframe to shuffle the gene locations
# reasign the same 'gene_ids' to match the expression df
# on all chrs takes ~ 10min
final_residual_exp = two_part_residualize(expression_df, base_covariates, peer_covariates)
shuffled_expression = final_residual_exp.sample(len(final_residual_exp))
shuffled_expression.index = final_residual_exp.index
null_cluster_df = call_clusters.get_clusters(expression_df_full, shuffled_expression, my_tissue_id)

# cluster density on per chrom basis for the null clusters
cluster_density_null = []
for i in np.arange(1,23,1):
    chr_gene_ids = expression_df_full[expression_df_full['#chr'] == f'chr{i}']['gene_id']
    cluster_density_null.append(null_cluster_df[null_cluster_df['Chromosome'] == i]['N_genes'].sum()/len(chr_gene_ids))

In [ ]:
# combine into a plot
cluster_density_df = pd.DataFrame({'Real gene locations': cluster_density, 'Shuffled gene locations':cluster_density_null})
# plot swarmplot
ax = sns.swarmplot(cluster_density_df,color='k', zorder=0, marker="$\circ$", ec="face", s=10, alpha=.8)
# plot boxplot
sns.boxplot(cluster_density_df, 
                 showcaps=True,boxprops={'facecolor':'None'},
                 showfliers=False, whiskerprops={'linewidth':1}, ax=ax)
ax.set_ylabel('Fraction genes in clusters')
ax.spines[['top','right']].set_visible(False)
plt.show()

### cross mappable clusters

In [ ]:
# remove cross mappable 
clusters_annotated = load_across_tissues(config, load_clusters_annotated)

fig, ax = plt.subplots(figsize=(8,5))
sns.histplot(clusters_annotated, x='num_cross_map', hue='has_cross_map', 
             bins=np.arange(0, clusters_annotated['num_cross_map'].max()),
             palette=('k', 'lightgrey'), hue_order=[True, False])

# customize the legend
legend = ax.get_legend()
legend.get_frame().set_visible(False)
legend.set_title("Removed from QTL analysis?")
# make the plot pretty
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlabel("Number genes in cluster with at least one cross-mappable XX-mer")
ax.set_ylabel("Number of clusters")

### cluster sharing across tissues

In [ ]:
# for each pair of genes, are they in a cluster in any other set of tisues?
clusters['gene_id'] = clusters['cluster_id'].str.split('_')
clusters['partner_gene_id'] = clusters['cluster_id'].str.split('_')f
clusters_exploded = clusters.explode('gene_id').explode('partner_gene_id')
# remove ones where genes are partnered to themselves 
clusters_exploded = clusters_exploded[~(clusters_exploded['gene_id'] == clusters_exploded['partner_gene_id'])]
# group by pairs of genes
gene_partners = clusters_exploded.groupby(['gene_id', 'partner_gene_id']).agg({'cluster_id':'unique', 'tissue_id':'unique'})

def get_pretty_tissue_id(tissue_id_list):
    return [tissue_id.replace('_', ' ') for tissue_id in tissue_id_list]
gene_partners['tissue_id_pretty'] = gene_partners['tissue_id'].apply(get_pretty_tissue_id)

In [ ]:
# make an upset plot showing pair sharing
upset = up.UpSet(up.from_memberships(gene_partners['tissue_id_pretty']), subset_size='count', min_subset_size=25, show_counts=False, 
                 sort_by="cardinality", facecolor="grey", totals_plot_elements=3)
for i in range(6):
    upset.style_subsets(min_degree=i, facecolor=sns.color_palette("Greys")[i])

# style the instersection plot
plot_result = upset.plot()
intersection_plot = plot_result["intersections"]
intersection_plot.set_ylabel("Shared gene pairs")
intersection_plot.set_xlim(-.8, intersection_plot.get_xlim()[1])
intersection_plot.set_ylim(0, intersection_plot.get_ylim()[1]+100)
intersection_plot.grid(False)

# style the totals plot
totals_plot = plot_result["totals"]
totals_plot.set_xlim((8000, 0))
totals_plot.grid(False)
totals_plot.set_xlabel("Total gene pairs")
# change totals plot colors to match gtex tissues
gtex_tissue_pal_df['tissue_id_pretty'] = gtex_tissue_pal_df['tissue_id'].str.replace('_', ' ')
for bar, label in zip(plot_result["totals"].patches, totals_plot.get_yticklabels()):
    bar.set_color(gtex_tissue_pal_df[gtex_tissue_pal_df['tissue_id_pretty'] == label.get_text()]['hex'].iloc[0])  # Set the color based on the palette

plt.show()